In [33]:
from collections import defaultdict
import os
os.chdir("/home/jovyan/work")

from src.utils import send_request

In [34]:
pbp = send_request("https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20151028-UTA-DET/playbyplay.json")

In [35]:
parsed_games = defaultdict(list)

In [36]:
def get_play_type(play):
    return (set(play.keys()) - set(["description", "playStatus"])).pop()

In [37]:
def _pass_play(play):
    pass

In [38]:
def unmarshall_jumpball(jumpball):
    result = {}
    
    # attributes
    result["name"] = jumpball["description"]
    result["wonBy"] = jumpball["jumpBall"]["wonBy"]
    
    # relationships
    result["homePlayerID"] = jumpball["jumpBall"]["homePlayer"]["id"]
    result["awayPlayerID"] = jumpball["jumpBall"]["awayPlayer"]["id"]
    result["secondsElapsed"] = jumpball["playStatus"]["secondsElapsed"]
    result["quarter"] = jumpball["playStatus"]["quarter"]
    
    parsed_games["jumpball"].append(result)

In [39]:
def unmarshall_fieldGoalAttempt(fieldGoalAttempt):
    result = {}
    
    # attributes
    result["name"] = fieldGoalAttempt["description"]
    result["points"] = fieldGoalAttempt["fieldGoalAttempt"]["points"]
    result["result"] = fieldGoalAttempt["fieldGoalAttempt"]["result"]
    result["shotType"] = fieldGoalAttempt["fieldGoalAttempt"]["shotType"]
    
    # relationships
    result["shootingPlayerID"] = fieldGoalAttempt["fieldGoalAttempt"]["shootingPlayer"]["id"]
    result["secondsElapsed"] = fieldGoalAttempt["playStatus"]["secondsElapsed"]
    result["quarter"] = fieldGoalAttempt["playStatus"]["quarter"]
    
    # fragile values
    if fieldGoalAttempt["fieldGoalAttempt"]["blockingPlayer"] == None:
        result["blockingPlayerID"] = -1
    else:
        result["blockingPlayerID"] = fieldGoalAttempt["fieldGoalAttempt"]["blockingPlayer"]["id"]
    
    if fieldGoalAttempt["fieldGoalAttempt"]["assistingPlayer"] == None:
        result["assistingPlayerID"] = -1
    else:
        result["assistingPlayerID"] = fieldGoalAttempt["fieldGoalAttempt"]["assistingPlayer"]["id"]
    
    parsed_games["fieldGoalAttempt"].append(result)

In [40]:
def unmarshall_turnover(turnover):
    result = {}
    
    # attributes
    result["name"] = turnover["description"]
    result["isStolen"] = turnover["turnover"]["isStolen"]
    result["type"] = turnover["turnover"]["type"]
    
    # relationships
    result["lostByPlayerID"] = turnover["turnover"]["lostByPlayer"]["id"]
    result["secondsElapsed"] = turnover["playStatus"]["secondsElapsed"]
    result["quarter"] = turnover["playStatus"]["quarter"]
    
    # fragile values
    if turnover["turnover"]["stolenByPlayer"] == None:
        result["stolenByPlayerID"] = -1
    else:
        result["stolenByPlayerID"] = turnover["turnover"]["stolenByPlayer"]["id"]
    
    parsed_games["turnover"].append(result)

In [41]:
def unmarshall_rebound(rebound):
    result = {}
    
    # attributes
    result["name"] = rebound["description"]
    result["type"] = rebound["rebound"]["type"]
    
    # relationships
    result["secondsElapsed"] = rebound["playStatus"]["secondsElapsed"]
    result["quarter"] = rebound["playStatus"]["quarter"]
    
    # fragile values
    if rebound["rebound"]["retrievingPlayer"] == None:
        result["retrievingPlayerID"] = -1
    else:
        result["retrievingPlayerID"] = rebound["rebound"]["retrievingPlayer"]["id"]
    
    parsed_games["rebound"].append(result)

In [42]:
def unmarshall_substitution(substitution):
    result = {}
    
    # attributes
    result["name"] = substitution["description"]
    
    # relationships
    result["secondsElapsed"] = substitution["playStatus"]["secondsElapsed"]
    result["quarter"] = substitution["playStatus"]["quarter"]
    
    # fragile values
    if substitution["substitution"]["incomingPlayer"] == None:
        result["incomingPlayerID"] = -1
    else:
        result["incomingPlayerID"] = substitution["substitution"]["incomingPlayer"]["id"]
    
    if substitution["substitution"]["outgoingPlayer"] == None:
        result["outgoingPlayerID"] = -1
    else:
        result["outgoingPlayer"] = substitution["substitution"]["outgoingPlayer"]["id"]
    
    parsed_games["substitution"].append(result)

In [43]:
def unmarshall_freeThrowAttempt(freeThrowAttempt):
    result = {}
    
    # attributes
    result["name"] = freeThrowAttempt["description"]
    result["attemptNum"] = freeThrowAttempt["freeThrowAttempt"]["attemptNum"]
    result["result"] = freeThrowAttempt["freeThrowAttempt"]["result"]
    result["totalAttempts"] = freeThrowAttempt["freeThrowAttempt"]["totalAttempts"]
    
    # relationships
    result["shootingPlayerID"] = freeThrowAttempt["freeThrowAttempt"]["shootingPlayer"]["id"]
    result["secondsElapsed"] = freeThrowAttempt["playStatus"]["secondsElapsed"]
    result["quarter"] = freeThrowAttempt["playStatus"]["quarter"]
    
    parsed_games["freeThrowAttempt"].append(result)

In [44]:
def unmarshall_violation(violation):
    result = {}
    
    # attributes
    result["name"] = violation["description"]
    result["teamOrPersonal"] = violation["violation"]["teamOrPersonal"]
    result["playerID"] = violation["violation"]["player"]["id"]
    
    # relationships
    result["teamAbbreviation"] = violation["violation"]["team"]["abbreviation"]
    result["teamAbbreviation"] = violation["violation"]["team"]["abbreviation"]
    result["secondsElapsed"] = violation["playStatus"]["secondsElapsed"]
    result["quarter"] = violation["playStatus"]["quarter"]
    
    parsed_games["violation"].append(result)

In [45]:
def unmarshall_foul(foul):
    result = {}
    
    # attributes
    result["name"] = foul["description"]
    result["teamAbbreviation"] = foul["foul"]["team"]["abbreviation"]
    
    # relationships
    result["secondsElapsed"] = foul["playStatus"]["secondsElapsed"]
    result["quarter"] = foul["playStatus"]["quarter"]
    
    # fragile values
    if foul["foul"]["penalizedPlayer"] == None:
        result["penalizedPlayerID"] = -1
    else:
        result["penalizedPlayerID"] = foul["foul"]["penalizedPlayer"]["id"]

    if foul["foul"]["drawnByPlayer"] == None:
        result["drawnByPlayerID"] = -1
    else:
        result["drawnByPlayerID"] = foul["foul"]["drawnByPlayer"]["id"]

    if  foul["foul"]["isPersonal"]:
        result["type"] = "personal"
    elif  foul["foul"]["isTechnical"]:
        result["type"] = "technical"
    elif  foul["foul"]["isFlagrant1"]:
        result["type"] = "flagrant1"
    elif  foul["foul"]["isFlagrant2"]:
        result["type"] = "flagrant2"
    else:
        result["type"] = "unnkown"

    parsed_games["foul"].append(result)

In [46]:
unmarshall = {
    "jumpBall": unmarshall_jumpball,
    "fieldGoalAttempt": unmarshall_fieldGoalAttempt,
    "turnover": unmarshall_turnover,
    "foul": unmarshall_foul,
    "rebound": unmarshall_rebound,
    "substitution": unmarshall_substitution,
    "freeThrowAttempt": unmarshall_freeThrowAttempt,
    "violation": unmarshall_violation
}

In [48]:
for play in pbp.get("plays"):
    play_type = get_play_type(play)
    unmarshall[play_type](play)

In [52]:
import pandas as pd

In [53]:
pd.DataFrame.from_records(parsed_games.get("turnover"))

,isStolen,lostByPlayerID,name,quarter,secondsElapsed,stolenByPlayerID,type
0,False,9207,Marc Morris Foul Turnover (P1.T1),1,31,-1,Foul
1,False,9503,Gobert Bad Pass Turnover (P1.T1),1,73,-1,Bad Pass
2,False,9506,Hayward Foul Turnover (P1.T2),1,290,-1,Foul
3,True,9213,Jackson Bad Pass Turnover (P1.T2),1,303,9503,Bad Pass
4,False,9203,Caldwell-Pope Out of Bounds Lost Ball Turnover...,1,397,-1,Lost Ball
5,True,9510,Hood Lost Ball Turnover (P1.T3),1,471,9400,Lost Ball
6,False,9505,Favors Traveling Turnover (P1.T4),1,541,-1,Traveling
7,True,9205,Blake Poss Lost Ball Turnover (P1.T4),2,17,9538,Lost Ball
8,False,9506,Hayward Bad Pass Turnover (P2.T5),2,546,-1,Bad Pass
9,False,9506,Hayward Foul Turnover (P3.T6),2,643,-1,Foul
